In [1]:
import pandas as pd
from io import BytesIO
import requests
from urllib.parse import urlencode
import urllib
from requests.auth import HTTPBasicAuth
from requests.exceptions import ChunkedEncodingError
import os
import json
import yadisk
from datetime import datetime, date, timedelta
import locale
from time import sleep
import shutil
import gc
import turbodbc
from turbodbc import connect
import gc
from pandas.api.types import is_string_dtype
import numpy as np
from sqlalchemy import create_engine
import pyodbc
import warnings


pd.options.mode.chained_assignment = None


# параметры подключения к нашей БД, в которую сохранить итоговые таблицы
server_local = 'REGRU-54856\MSSQLSERVER01'
database_local = 'custom_tables_NL_2208' #'custom_tables_NL_2208' - БД для тестовой загрузки / 'NL_db_27082024' - БД для дашбодра НЛ
conn_str_local = 'DRIVER={SQL Server};SERVER='+server_local+';DATABASE='+database_local+';'

# указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\32_NL_Yandex_Disk'
file_name = 'yandex_token.txt'

# сохраняем токен в отдельную переменную
yandex_token = open(os.path.join(file_path, file_name), encoding='utf-8').read()
# формируем заголовки для дальнейших запросов
headers = {
    'Content-Type': 'application/json', 
    'Accept': 'application/json', 
    'Authorization': f'OAuth {yandex_token}'
}

# указываем путь к основной папке, в которой храняться папки с флайтами
main_folder = '/NL_отчеты_для_дашборда'

# УРЛ для операций над опубликованными папками и файлами на Яндекс Диске
base_public_url = 'https://cloud-api.yandex.net/v1/disk/public/resources?'  
public_key = 'https://disk.yandex.ru/d/2iYr8zMm9Qgeug' # обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета

download_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' # УРЛ для скачивания эксель файлов из Яндекс Диска
delete_url = 'https://cloud-api.yandex.net/v1/disk/resources?' # УРЛ для удаления папок и файлов на Яндекс Диске

final_url = base_public_url + urlencode(dict(public_key=public_key))  # Формируем УРЛ для получения списка папок и файлов
res = requests.get(final_url)  # Отправлем запрос на Яндекс диск, чтобы получить название папок и public key для скачивания файлов

print(res.status_code)

yandex_folders = res.json() # парсим ответ

200


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)

# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)

pd.set_option('display.max_rows', None)

pd.options.mode.chained_assignment = None

In [3]:
# Создаем функцию для получения содержимого Яндекс диска
#  на входе принимаем 
# - адрес запроса для скачивания файла
# - public key, который мы получаем из ответа Яндекс при запросе информации по конкретной папке
# - адрес папки
# - лимит файлов (максимальное кол-во 80)
def get_yandex_disk_responce(request_url, public_key, folder_path, limit=80):
    res_url = request_url + urlencode(dict(public_key=public_key, path=folder_path, limit=limit)) # формируем строку запроса
    res = requests.get(res_url) # отправляем запрос на сервер
    yandex_responce = res.json() # получаем ответ и преобразуем его в json

    return yandex_responce

In [4]:
public_key = yandex_folders['public_key']  # из ответа Яндекс забираем public_key, чтобы использовать его для скачивания файлов

for i in range(len(yandex_folders['_embedded']['items'])): # через цикл проходим по ответу Яндекса и забираем названия вложенных папок
    file_type = yandex_folders['_embedded']['items'][i]['type']
    if file_type=='dir':   # если находим файлы с типом dir (папка), то забираем путь к этой папке
        folder_path = yandex_folders['_embedded']['items'][i]['path']
        print(folder_path)
        yandex_responce = get_yandex_disk_responce(base_public_url, public_key, folder_path) # отправляем запрос, чтобы получить содержимое папки

        # Через цикл проходим по папке с файлами
        # Нас интересуют файлы эксель. Причем каждая экселька будет парситься по своему, т.к. они относятся к разным рекламным площадкам
        
        # Проходим через цикл по содержимому папки (отдельный флайт)
        for i in range(len(yandex_responce['_embedded']['items'])):
            file_info = yandex_responce['_embedded']['items'][i]
            if file_info['type']=='file':  # если документ является фалйом(не папкой или изображением), то забираем его название 
                file_name = file_info['name'] # сохраняем название файла
                if 'xls' in file_name: # еслит тип файла является xlsx, то уберем расширение и будем его использовать в качесвте названия отчета
                    file_path = file_info['path']
                    
                    report_name = '.'.join(file_name.split('.')[:-1]) # убираем .xlsx из названия файла
                    print(report_name)
                    res_file_link = get_yandex_disk_responce(download_url, public_key, file_path) # получаем ссылку на скачивание отчета
                    download_response = requests.get(res_file_link['href'])

/01_new_sources_files
/02_regular_reports
gnezdo_banner


In [5]:
data_link = download_response.content

In [6]:
sheet_names = pd.ExcelFile(BytesIO(data_link))

In [7]:
sheet_names.sheet_names

['суммарная статистика',
 'Наци...лотерея-Наци...телефоне',
 'Наци...лотерея-Наци...озрения ',
 'Наци...лотерея-Наци...еловека ',
 'Наци...лотерея-Наци...татуэтка',
 'Наци...лотерея-Наци...человека']

In [8]:
report_type = 'test'

In [9]:
tmp_banner_dict = {}

cols_range = 'A:E' # задаем диапазон полей, которые нам нужны
# создаем список с названиями полей

In [117]:
for name in sheet_names.sheet_names[1:]:
    # передаем название листа для парсинга, диапазон колонок, которые нам нужны и заранее подготовленный списко названий полей
    df = pd.read_excel(BytesIO(data_link), sheet_name=name, usecols=cols_range, header=None) 
    df = df.fillna(0)  #заполяем пустые строки, чтобы затем их удалить

    base_text = df[0][3].lower().strip() # Название креатива(продукта) находится в столбике А в 4 строке - указывается в самом конце в скобочках
    base_text = re.sub('[_!;:()-]+', ' ', base_text )
    base_text = re.sub(' +', ' ', base_text)
    
    start_index = str(base_text).find('креатив') # Забираем индекс, с которого начинается название креатива
    product_name = str(base_text)[start_index:] # сохраняем название креатива

    start_index = list(df[df[0]=='Дата'].index)[0]  # берем индекс начала таблицы с данными
    end_index = list(df[df[0]=='Всего'].index)[0] # берем индекс окончания таблицы с данными

    df.columns = df.iloc[start_index].str.lower().str.strip().str.replace('\n', ' ') # забираем название полей из файла
    df = df.iloc[start_index+1:end_index] # оставляем строки с данными, которые нам нужны
     # привоодим названия к единому стандарту
    df = df.rename(columns={'дата': 'date', 'охват': 'reach', 'показы': 'impressions', 'видимые показы': 'views', 'переходы': 'clicks'}) 
    df = df[['date', 'reach', 'impressions', 'views', 'clicks']] # оставляем только нужные поля

    df['source'] = 'gnezdo' #добавляем название источника
    df['format_type'] = 'banner' # добавляем статичое поле с название Типа формата рекламы (Видео/Баннер)
    df['date'] = df['date'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y').date().strftime('%Y-%m-%d'))#pd.to_datetime(df['date']).dt.date  # приводим в формат даты
    
    # df.insert(1, 'product', product_name, True) # добавляем статичное поле с названием продукта
    df['product'] = product_name # добавляем статичное поле с названием продукта
    df['product'] = df['product'].str.lower().str.strip()
    df['report_type'] = report_type
    # сохраняем датаФрейм во временный словарь 
    # ключ - это название продукта (15s, 6s и тд)
    tmp_banner_dict[name] = df

In [118]:
tmp_banner_dict

{'Наци...лотерея-Наци...телефоне': 5         date  reach impressions  views clicks  source format_type  \
 6   2024-09-16  21653      136722  29026    296  gnezdo      banner   
 7   2024-09-17  19231      137359  24971    297  gnezdo      banner   
 8   2024-09-18  23051      206278  28061    303  gnezdo      banner   
 9   2024-09-19  19598      175707  27675    298  gnezdo      banner   
 10  2024-09-20  27445      201969  39990    305  gnezdo      banner   
 11  2024-09-21  24387      253460  35118    304  gnezdo      banner   
 12  2024-09-22      0           0      0      0  gnezdo      banner   
 
 5                                  product report_type  
 6   креатив 3 колесо обозрения на телефоне        test  
 7   креатив 3 колесо обозрения на телефоне        test  
 8   креатив 3 колесо обозрения на телефоне        test  
 9   креатив 3 колесо обозрения на телефоне        test  
 10  креатив 3 колесо обозрения на телефоне        test  
 11  креатив 3 колесо обозрения на телеф

In [113]:
df

5,date,reach,показы,views,clicks
6,16.09.2024,0,0,0,0
7,17.09.2024,0,0,0,0
8,18.09.2024,0,0,0,0
9,19.09.2024,0,0,0,0
10,20.09.2024,0,0,0,0
11,21.09.2024,0,0,0,0
12,22.09.2024,52427,422985,87616,191


In [102]:
df

5,дата,охват,показы,видимые\nпоказы,переходы
6,16.09.2024,0,0,0,0
7,17.09.2024,0,0,0,0
8,18.09.2024,0,0,0,0
9,19.09.2024,0,0,0,0
10,20.09.2024,0,0,0,0
11,21.09.2024,0,0,0,0
12,22.09.2024,52427,422985,87616,191


'национальная лотерея национальная лотерея день рождения "мечталлион" креатив 4 торт и 4 человека'

'креатив  4 торт и 4 человека'

In [95]:
df.head(6)

,0,1,2,3,4
0,0,0,0,0,0
1,Статистика переходов с 16.09.2024 по 22.09.2024,0,0,0,0
2,по дням,0,0,0,0
3,"Национальная лотерея - Национальная лотерея_ День рождения ""Мечталлион""_ Креатив_ 4_торт и 4 человека",0,0,0,0
4,0,0,0,0,0
5,Дата,Охват,Показы,Видимые\nпоказы,Переходы


In [ ]:
df_tmp = df.iloc[start_index+1:end_index] # оставляем строки с данными, которые нам нужны

In [ ]:
# источник Gnezdo
# тип отчета Баннерная реклама
# Функция для обработки статистики по дням для обычных размещений 
# Каждый креатив на отдельном листе

def get_gnezdo_banner_report(data_link, report_type):
    tmp_banner_dict = {}
    
    cols_range = 'A:E' # задаем диапазон полей, которые нам нужны
    # создаем список с названиями полей
    cols_name = ['date', 'reach', 'impressions', 'views', 'clicks']
    
    sheet_names = pd.ExcelFile(BytesIO(data_link))
    # проходим через цикл по списку названий листов
    # sheets_list = sheet_names.sheet_names[1:]
    for name in sheet_names.sheet_names[1:]:
        # передаем название листа для парсинга, диапазон колонок, которые нам нужны и заранее подготовленный списко названий полей
        df = pd.read_excel(BytesIO(data_link), sheet_name=name, usecols=cols_range, header=None, names=cols_name) 
        df = df.fillna(0)  #заполяем пустые строки, чтобы затем их удалить

        base_text = df['date'][3] # Название креатива(продукта) находится в столбике А в 4 строке - указывается в самом конце в скобочках
        start_index = str(base_text).find(' (') + 2 # Забираем индекс, с которого начинается название креатива
        end_index = str(base_text).find(') ') # Забираем индекс, которым заканчивается назание креатива
        product_name = str(base_text)[start_index:end_index] # сохраняем название креатива

        start_index = list(df[df['date']=='Дата'].index)[0]  # берем индекс начала таблицы с данными
        end_index = list(df[df['date']=='Всего'].index)[0] # берем индекс окончания таблицы с данными

        df = df.iloc[start_index+1:end_index] # оставляем строки с данными, которые нам нужны
        df['source'] = 'gnezdo' #добавляем название источника
        df['format_type'] = 'banner' # добавляем статичое поле с название Типа формата рекламы (Видео/Баннер)
        df['date'] = df['date'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y').date().strftime('%Y-%m-%d'))#pd.to_datetime(df['date']).dt.date  # приводим в формат даты
        
        df.insert(1, 'product', product_name, True) # добавляем статичное поле с названием продукта
        df['product'] = df['product'].str.lower()
        df['report_type'] = report_type
        # сохраняем датаФрейм во временный словарь 
        # ключ - это название продукта (15s, 6s и тд)
        tmp_banner_dict[name] = df

    return pd.concat(tmp_banner_dict, ignore_index=True)